In [10]:
import json
import os
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
import time
from tqdm import tqdm
import glob

def wait_for_qdrant(max_retries=30):
    """Ждет пока Qdrant запустится"""
    print("Ожидаем запуск Qdrant...")
    for i in range(max_retries):
        try:
            client = QdrantClient(host="localhost", port=6333)
            client.get_collections()
            print("Qdrant готов!")
            return True
        except Exception:
            if i % 5 == 0:
                print(f"   Попытка {i+1}/{max_retries}...")
            time.sleep(2)
    raise Exception("Qdrant не запустился")

def find_all_json_files(data_folder="data"):
    """Находит все JSON файлы в папке data"""
    json_pattern = os.path.join(data_folder, "*.json")
    json_files = glob.glob(json_pattern)
    
    print(f"🔍 Поиск JSON файлов в папке {data_folder}:")
    for file in json_files:
        print(f"   📄 {file}")
    
    return json_files

def load_all_json_data_with_ids(data_folder="data"):
    """Загружает все JSON файлы из папки data с уникальными ID"""
    json_files = find_all_json_files(data_folder)
    
    if not json_files:
        print(f"❌ В папке {data_folder} не найдено JSON файлов")
        return None
    
    all_data = []
    id_offset = 0
    
    for json_file in json_files:
        print(f"\n📂 Загружаем файл: {json_file}")
        
        if not os.path.exists(json_file):
            print(f"   ❌ Файл {json_file} не найден")
            continue
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Добавляем уникальные ID для каждого файла
            for item in data:
                item['unique_id'] = id_offset + item.get('chunk_id', 0)
                item['file_source'] = os.path.basename(json_file)
            
            print(f"   ✅ Успешно загружено {len(data)} чанков (ID: {id_offset} - {id_offset + len(data) - 1})")
            all_data.extend(data)
            id_offset += len(data)  # Смещение для следующего файла
            
        except Exception as e:
            print(f"   ❌ Ошибка загрузки {json_file}: {e}")
            continue
    
    print(f"\n📊 Всего загружено чанков из всех файлов: {len(all_data)}")
    print(f"📊 Диапазон ID: 0 - {len(all_data) - 1}")
    return all_data

def setup_json_collection(collection_name="json_chunks"):
    """Создает коллекцию для JSON данных"""
    client = QdrantClient(host="localhost", port=6333)
    
    try:
        # Пробуем удалить старую коллекцию
        client.delete_collection(collection_name)
        print(f"🗑️ Удалена старая коллекция {collection_name}")
    except:
        pass
    
    # Временно создаем с размером 384, потом пересоздадим с правильным
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )
    print(f"🆕 Создана коллекция {collection_name}")
    
    return collection_name

def check_collection_info(collection_name="json_chunks"):
    """Проверяет информацию о коллекции"""
    client = QdrantClient(host="localhost", port=6333)
    
    try:
        info = client.get_collection(collection_name)
        print(f"📊 Коллекция '{collection_name}':")
        print(f"   • Точек: {info.points_count}")
        print(f"   • Размер вектора: {info.config.params.vectors.size}")
        
        # Показываем примеры из разных источников
        points = client.scroll(
            collection_name=collection_name,
            limit=5
        )[0]
        
        print("\n📝 Примеры данных:")
        for i, point in enumerate(points):
            print(f"   Пример {i+1}:")
            print(f"     ID: {point.id}")
            print(f"     Источник файла: {point.payload.get('file_source', 'unknown')}")
            print(f"     URL: {point.payload.get('source', '')}")
            print(f"     Заголовок: {point.payload.get('title', '')[:50]}...")
            print()
            
    except Exception as e:
        print(f"❌ Ошибка проверки коллекции: {e}")

def verify_collection_count(collection_name="json_chunks", expected_count=20732):
    """Проверяет количество точек в коллекции"""
    client = QdrantClient(host="localhost", port=6333)
    
    try:
        info = client.get_collection(collection_name)
        actual_count = info.points_count
        
        print(f"\n🔍 ПРОВЕРКА КОЛИЧЕСТВА:")
        print(f"   Ожидалось: {expected_count} точек")
        print(f"   Фактически: {actual_count} точек")
        
        if actual_count == expected_count:
            print(f"   ✅ ВСЕ ТОЧКИ ЗАГРУЖЕНЫ!")
        else:
            print(f"   ❌ ПРОБЛЕМА: не хватает {expected_count - actual_count} точек")
            
        return actual_count
        
    except Exception as e:
        print(f"❌ Ошибка проверки количества: {e}")
        return 0

def process_and_upload_json_fixed(data, collection_name="json_chunks", batch_size=100):
    """Обрабатывает и загружает JSON данные в Qdrant с уникальными ID"""
    client = QdrantClient(host="localhost", port=6333)
    
    if not data:
        print("❌ Нет данных для загрузки")
        return 0
    
    # Определяем размер вектора по первому элементу
    first_item = data[0]
    if 'embedding' in first_item and first_item['embedding']:
        vector_size = len(first_item['embedding'])
        print(f"📏 Размер вектора: {vector_size}")
        
        # Пересоздаем коллекцию с правильным размером вектора
        try:
            client.delete_collection(collection_name)
        except:
            pass
            
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
        )
        print(f"🔄 Пересоздана коллекция с размером вектора: {vector_size}")
    else:
        print("❌ В данных нет эмбеддингов")
        return 0
    
    points = []
    successful_chunks = 0
    total_loaded = 0
    skipped_chunks = 0
    
    print("📤 Подготавливаем данные для загрузки...")
    
    for item in tqdm(data, desc="Обработка чанков"):
        try:
            # Проверяем наличие обязательных полей
            if 'embedding' not in item or not item['embedding']:
                skipped_chunks += 1
                continue
            
            # Проверяем размер эмбеддинга
            if len(item['embedding']) != vector_size:
                print(f"⚠️ Неправильный размер эмбеддинга: {len(item['embedding'])} (ожидалось: {vector_size})")
                skipped_chunks += 1
                continue
            
            # Используем УНИКАЛЬНЫЙ ID вместо chunk_id
            unique_id = item.get('unique_id', len(points) + total_loaded)
            
            # Создаем точку для Qdrant
            point = {
                "id": unique_id,
                "vector": item["embedding"],
                "payload": {
                    "text": item.get("text", ""),
                    "title": item.get("title", ""),
                    "source": item.get("source", ""),
                    "word_count": item.get("word_count", 0),
                    "chunk_id": item.get("chunk_id", 0),
                    "file_source": item.get("file_source", "unknown"),
                    "unique_id": unique_id
                }
            }
            
            # Добавляем все остальные поля из JSON в payload
            for key, value in item.items():
                if key not in ["embedding", "chunk_id", "unique_id"] and key not in point["payload"]:
                    point["payload"][key] = value
            
            points.append(point)
            successful_chunks += 1
            
            # Загружаем батчами
            if len(points) >= batch_size:
                print(f"🚀 Загружаем батч из {len(points)} точек...")
                try:
                    client.upsert(
                        collection_name=collection_name,
                        points=points
                    )
                    total_loaded += len(points)
                    print(f"✅ Успешно загружено {len(points)} чанков (всего: {total_loaded})")
                    points = []  # Очищаем points после успешной загрузки
                except Exception as e:
                    print(f"❌ Ошибка загрузки батча: {e}")
                    # Пробуем уменьшить размер батча
                    if batch_size > 10:
                        batch_size = max(10, batch_size // 2)
                        print(f"🔄 Уменьшаем размер батча до {batch_size}")
                    continue
            
        except Exception as e:
            print(f"⚠️ Ошибка обработки чанка ID {item.get('unique_id', 'unknown')}: {e}")
            skipped_chunks += 1
            continue
    
    # Загружаем оставшиеся точки
    if points:
        print(f"🚀 Загружаем последний батч из {len(points)} точек...")
        try:
            client.upsert(
                collection_name=collection_name,
                points=points
            )
            total_loaded += len(points)
            print(f"✅ Успешно загружено {len(points)} чанков (всего: {total_loaded})")
        except Exception as e:
            print(f"❌ Ошибка загрузки последнего батча: {e}")
    
    print(f"📊 Статистика загрузки:")
    print(f"   • Всего обработано чанков: {len(data)}")
    print(f"   • Успешно загружено: {total_loaded}")
    print(f"   • Пропущено: {skipped_chunks}")
    
    return total_loaded

def main():
    """Основная функция"""
    print("=" * 60)
    print("ЗАГРУЗКА ВСЕХ JSON ДАННЫХ В QDRANT (ИСПРАВЛЕННАЯ)")
    print("=" * 60)
    
    # Ждем Qdrant
    wait_for_qdrant()
    
    # Создаем коллекцию
    collection_name = setup_json_collection()
    
    # Загружаем ВСЕ JSON данные из папки data с УНИКАЛЬНЫМИ ID
    all_data = load_all_json_data_with_ids("data")
    
    if not all_data:
        print("❌ Не удалось загрузить данные")
        return
    
    # Обрабатываем и загружаем данные с исправленными ID
    chunks_loaded = process_and_upload_json_fixed(all_data, collection_name)
    
    # Итоги
    print("\n" + "=" * 60)
    print("ЗАГРУЗКА ЗАВЕРШЕНА!")
    print("=" * 60)
    print(f"📈 Итоги:")
    print(f"   • Всего чанков в JSON файлах: {len(all_data)}")
    print(f"   • Успешно загружено в Qdrant: {chunks_loaded}")
    print(f"   • Коллекция: '{collection_name}'")
    
    # Проверяем коллекцию
    check_collection_info(collection_name)
    
    # ВЕРИФИКАЦИЯ количества точек
    verify_collection_count(collection_name, len(all_data))

if __name__ == "__main__":
    main()

ЗАГРУЗКА ВСЕХ JSON ДАННЫХ В QDRANT (ИСПРАВЛЕННАЯ)
Ожидаем запуск Qdrant...
Qdrant готов!
🗑️ Удалена старая коллекция json_chunks
🆕 Создана коллекция json_chunks
🔍 Поиск JSON файлов в папке data:
   📄 data\chunks_with_embeddings.json
   📄 data\chunks_with_embeddings_2.json

📂 Загружаем файл: data\chunks_with_embeddings.json
   ✅ Успешно загружено 11420 чанков (ID: 0 - 11419)

📂 Загружаем файл: data\chunks_with_embeddings_2.json
   ✅ Успешно загружено 9312 чанков (ID: 11420 - 20731)

📊 Всего загружено чанков из всех файлов: 20732
📊 Диапазон ID: 0 - 20731
📏 Размер вектора: 1024
🔄 Пересоздана коллекция с размером вектора: 1024
📤 Подготавливаем данные для загрузки...


Обработка чанков:   0%|          | 0/20732 [00:00<?, ?it/s]

🚀 Загружаем батч из 100 точек...


Обработка чанков:   0%|          | 100/20732 [00:00<01:27, 235.20it/s]

✅ Успешно загружено 100 чанков (всего: 100)
🚀 Загружаем батч из 100 точек...
✅ Успешно загружено 100 чанков (всего: 200)


...

Обработка чанков:  99%|█████████▉| 20600/20732 [01:00<00:00, 162.46it/s]

✅ Успешно загружено 100 чанков (всего: 20600)
🚀 Загружаем батч из 100 точек...


Обработка чанков: 100%|██████████| 20732/20732 [01:00<00:00, 340.96it/s]

✅ Успешно загружено 100 чанков (всего: 20700)
🚀 Загружаем последний батч из 32 точек...


✅ Успешно загружено 32 чанков (всего: 20732)
📊 Статистика загрузки:
   • Всего обработано чанков: 20732
   • Успешно загружено: 20732
   • Пропущено: 0

ЗАГРУЗКА ЗАВЕРШЕНА!
📈 Итоги:
   • Всего чанков в JSON файлах: 20732
   • Успешно загружено в Qdrant: 20732
   • Коллекция: 'json_chunks'
📊 Коллекция 'json_chunks':
   • Точек: 24664
   • Размер вектора: 1024

📝 Примеры данных:
   Пример 1:
     ID: 0
     Источник файла: chunks_with_embeddings.json
     URL: https://alfabank.ru/
     Заголовок: Альфа-Банк - кредитные и дебетовые карты, кредиты ...

   Пример 2:
     ID: 1
     Источник файла: chunks_with_embeddings.json
     URL: https://alfabank.ru/
     Заголовок: Альфа-Банк - кредитные и дебетовые карты, кредиты ...

   Пример 3:
     ID: 2
     Источник файла: chunks_with_embeddings.json
     URL: https://alfabank.ru/a-club/
     Заголовок: А-Клуб. Деньги имеют значение...

   Пример 4:
     ID: 3
     Источник файла: chunks_with_embeddings.json
     URL: https://alfabank.ru/a-club

# Создание сабмита

# Создание сабмита

In [11]:
import pandas as pd
import json
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import os
import traceback
from tqdm import tqdm
import hashlib
import re

class ImprovedBatchSearch:
    def __init__(self, model_path, collection_name="json_chunks"):
        try:
            self.client = QdrantClient(host="localhost", port=6333)
            self.collection_name = collection_name
            
            print(f"Загружаем локальную модель из: {model_path}")
            
            if not os.path.exists(model_path):
                print(f"❌ ОШИБКА: Путь '{model_path}' не существует!")
                return
            
            print("⏳ Загрузка модели...")
            self.model = SentenceTransformer(model_path, device='cpu')
            print("✅ Модель загружена!")
            
        except Exception as e:
            print(f"❌ Ошибка при инициализации: {e}")
            self.model = None
    
    def get_embedding(self, text):
        """Создает эмбеддинг для текста"""
        if self.model is None:
            return None
        return self.model.encode([text])[0].tolist()
    
    def is_good_result(self, title, text, url):
        """Проверяет, является ли результат хорошим"""
        # Плохие заголовки
        bad_titles = [
            'Карта сайта', 'Ограничения', 'Комиссия за квази-кэш', 
            'dogovor_x5', 'Без названия', 'test', 'Оставить заявку',
            'Альфа-Банк: страница не найдена'
        ]
        
        # Плохие URL паттерны
        bad_url_patterns = [
            r'.*\.pdf$', r'.*alfa_[\d_]+\.pdf$', r'.*ct-[\d_]+\.pdf$',
            r'.*private_[\d_]+\.pdf$', r'.*current_[\d_]+\.pdf$',
            r'.*balance_[\d_]+\.pdf$', r'.*airplane_[\d_]+\.pdf$'
        ]
        
        # Проверяем заголовок
        if any(bad_title in title for bad_title in bad_titles):
            return False
            
        # Проверяем URL
        if any(re.match(pattern, url) for pattern in bad_url_patterns):
            return False
            
        # Проверяем текст
        if len(text.strip()) < 30:
            return False
            
        # Проверяем, что текст не состоит только из повторяющихся фраз
        words = text.split()
        if len(set(words)) < 5:  # Если меньше 5 уникальных слов
            return False
            
        return True
    
    def search_high_quality_results(self, query, limit=5):
        """Поиск ВЫСОКОКАЧЕСТВЕННЫХ результатов"""
        if self.model is None:
            return []
            
        query_embedding = self.get_embedding(query)
        
        if query_embedding is None:
            return []
        
        # Ищем значительно больше результатов для лучшего выбора
        results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            limit=limit * 20  # Ищем в 10 раз больше
        )
        
        # Фильтруем только качественные результаты
        high_quality_results = []
        seen_urls = set()
        seen_titles = set()
        
        for result in results.points:
            if len(high_quality_results) >= limit:
                break
                
            url = result.payload.get('source', '')
            title = result.payload.get('title', '').strip()
            text = result.payload.get('text', '').strip()
            
            # Пропускаем дубликаты
            if url in seen_urls or title in seen_titles:
                continue
                
            # Проверяем качество результата
            if self.is_good_result(title, text, url):
                seen_urls.add(url)
                seen_titles.add(title)
                
                high_quality_results.append({
                    'score': result.score,
                    'title': title,
                    'url': url,
                    'text': text,
                    'chunk_id': result.payload.get('chunk_id', ''),
                    'word_count': result.payload.get('word_count', '')
                })
        
        # Если не нашли достаточно качественных результатов, пробуем менее строгий фильтр
        if len(high_quality_results) < limit:
            return self.search_with_broader_filter(query, limit, high_quality_results)
        
        return high_quality_results
    
    def search_with_broader_filter(self, query, limit, existing_results):
        """Поиск с более широкими критериями"""
        if self.model is None:
            return existing_results
            
        query_embedding = self.get_embedding(query)
        
        if query_embedding is None:
            return existing_results
        
        # Ищем дополнительные результаты
        additional_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            limit=limit * 10
        )
        
        seen_urls = {result['url'] for result in existing_results}
        seen_titles = {result['title'] for result in existing_results}
        
        # Менее строгие критерии
        mild_bad_titles = ['Карта сайта', 'Без названия', 'test']
        
        for result in additional_results.points:
            if len(existing_results) >= limit:
                break
                
            url = result.payload.get('source', '')
            title = result.payload.get('title', '').strip()
            text = result.payload.get('text', '').strip()
            
            # Более мягкие критерии
            if (url not in seen_urls and 
                title not in seen_titles and
                not any(bad in title for bad in mild_bad_titles) and
                len(text) > 20 and
                not re.match(r'.*\.pdf$', url)):
                
                seen_urls.add(url)
                seen_titles.add(title)
                
                existing_results.append({
                    'score': result.score,
                    'title': title,
                    'url': url,
                    'text': text,
                    'chunk_id': result.payload.get('chunk_id', ''),
                    'word_count': result.payload.get('word_count', '')
                })
        
        return existing_results

def create_improved_csv_links_only(results, output_csv="search_results_improved_links.csv"):
    """Создает CSV только с номерами запросов и 5 качественными ссылками"""
    
    flat_data = []
    
    for item in results:
        q_id = item['q_id']
        
        # Создаем базовую запись
        record = {
            'query_id': q_id,
            'query': item['query']
        }
        
        # Добавляем только качественные ссылки
        if item.get('results'):
            for i, result in enumerate(item['results']):
                if i < 5:  # Максимум 5 ссылок
                    record[f'link_{i+1}'] = result['url']
                    record[f'title_{i+1}'] = result['title']
                    record[f'score_{i+1}'] = result['score']
        
        # Заполняем оставшиеся пустые поля
        results_count = len(item.get('results', []))
        for i in range(results_count, 5):
            record[f'link_{i+1}'] = ''
            record[f'title_{i+1}'] = ''
            record[f'score_{i+1}'] = 0.0
        
        flat_data.append(record)
    
    # Создаем DataFrame и СОРТИРУЕМ по query_id
    df_links = pd.DataFrame(flat_data)
    df_links = df_links.sort_values('query_id').reset_index(drop=True)
    
    # Сохраняем в CSV
    df_links.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"✅ Улучшенный CSV со ссылками сохранен как: {output_csv}")
    print(f"📊 Размер: {len(df_links)} строк × {len(df_links.columns)} колонок")
    print(f"🔢 Отсортировано по query_id: {df_links['query_id'].min()} - {df_links['query_id'].max()}")
    
    return df_links

def create_improved_excel_with_texts(results, output_excel="search_results_improved_texts.xlsx"):
    """Создает Excel с полной информацией только качественных результатов"""
    
    excel_data = []
    
    for item in results:
        q_id = item['q_id']
        query = item['query']
        
        # Создаем базовую запись
        record = {
            'query_id': q_id,
            'query': query,
            'results_found': len(item.get('results', []))
        }
        
        # Добавляем полную информацию по каждой качественной ссылке
        if item.get('results'):
            for i, result in enumerate(item['results']):
                if i < 5:  # Максимум 5 результатов
                    record[f'link_{i+1}'] = result['url']
                    record[f'title_{i+1}'] = result['title']
                    record[f'score_{i+1}'] = result['score']
                    record[f'text_{i+1}'] = result['text']
                    record[f'word_count_{i+1}'] = result.get('word_count', '')
        
        # Заполняем оставшиеся пустые поля
        results_count = len(item.get('results', []))
        for i in range(results_count, 5):
            record[f'link_{i+1}'] = ''
            record[f'title_{i+1}'] = ''
            record[f'score_{i+1}'] = 0.0
            record[f'text_{i+1}'] = ''
            record[f'word_count_{i+1}'] = ''
        
        excel_data.append(record)
    
    # Создаем DataFrame и СОРТИРУЕМ по query_id
    df_excel = pd.DataFrame(excel_data)
    df_excel = df_excel.sort_values('query_id').reset_index(drop=True)
    
    # Сохраняем в Excel
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        df_excel.to_excel(writer, sheet_name='Search Results', index=False)
        
        # Настраиваем ширину колонок
        worksheet = writer.sheets['Search Results']
        worksheet.column_dimensions['A'].width = 10  # query_id
        worksheet.column_dimensions['B'].width = 50  # query
        worksheet.column_dimensions['C'].width = 12  # results_found
        
        # Настраиваем ширину для остальных колонок
        for col in range(4, len(df_excel.columns) + 1):
            col_letter = chr(64 + col) if col <= 26 else 'A' + chr(64 + col - 26)
            if 'text' in df_excel.columns[col-1]:
                worksheet.column_dimensions[col_letter].width = 60
            elif 'link' in df_excel.columns[col-1]:
                worksheet.column_dimensions[col_letter].width = 35
            elif 'title' in df_excel.columns[col-1]:
                worksheet.column_dimensions[col_letter].width = 25
            else:
                worksheet.column_dimensions[col_letter].width = 12
    
    print(f"✅ Улучшенный Excel с текстами сохранен как: {output_excel}")
    print(f"📊 Размер: {len(df_excel)} строк × {len(df_excel.columns)} колонок")
    print(f"🔢 Отсортировано по query_id: {df_excel['query_id'].min()} - {df_excel['query_id'].max()}")
    
    return df_excel

def process_dataframe_improved(df, model_path, sample_size=None):
    """Обрабатывает DataFrame с улучшенным поиском"""
    
    searcher = ImprovedBatchSearch(model_path)
    if searcher.model is None:
        return None
    
    if sample_size and sample_size < len(df):
        df_sample = df.sample(sample_size, random_state=42)
        print(f"🧪 Обрабатываем выборку из {sample_size} запросов")
    else:
        df_sample = df
        print(f"📊 Обрабатываем все {len(df)} запросов")
    
    # СОРТИРУЕМ исходный DataFrame по q_id перед обработкой
    df_sample = df_sample.sort_values('q_id').reset_index(drop=True)
    print(f"🔢 Исходные данные отсортированы по q_id: {df_sample['q_id'].min()} - {df_sample['q_id'].max()}")
    
    results = []
    quality_stats = {
        'total_queries': 0,
        'queries_with_5_results': 0,
        'queries_with_3_plus_results': 0,
        'queries_with_0_results': 0,
        'total_quality_results': 0
    }
    
    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Улучшенный поиск"):
        query = row['query']
        q_id = row['q_id']
        
        try:
            search_results = searcher.search_high_quality_results(query, limit=5)
            
            result_item = {
                'q_id': q_id,
                'query': query,
                'results': search_results,
                'results_count': len(search_results)
            }
            
            results.append(result_item)
            
            # Статистика качества
            results_count = len(search_results)
            quality_stats['total_quality_results'] += results_count
            
            if results_count == 5:
                quality_stats['queries_with_5_results'] += 1
            elif results_count >= 3:
                quality_stats['queries_with_3_plus_results'] += 1
            elif results_count == 0:
                quality_stats['queries_with_0_results'] += 1
                
        except Exception as e:
            print(f"❌ Ошибка для запроса {q_id}: {e}")
            results.append({
                'q_id': q_id,
                'query': query,
                'results': [],
                'results_count': 0,
                'error': str(e)
            })
            quality_stats['queries_with_0_results'] += 1
    
    quality_stats['total_queries'] = len(results)
    
    # СОРТИРУЕМ результаты по q_id перед сохранением
    results = sorted(results, key=lambda x: x['q_id'])
    print(f"🔢 Результаты отсортированы по q_id: {results[0]['q_id']} - {results[-1]['q_id']}")
    
    # Создаем оба файла
    print("\n💾 Сохраняем улучшенные результаты...")
    
    # 1. CSV только со ссылками
    df_csv = create_improved_csv_links_only(results)
    
    # 2. Excel с полной информацией
    df_excel = create_improved_excel_with_texts(results)
    
    # Детальная статистика
    print(f"\n📊 ДЕТАЛЬНАЯ СТАТИСТИКА КАЧЕСТВА:")
    print(f"   Обработано запросов: {quality_stats['total_queries']}")
    print(f"   Запросов с 5 качественными результатами: {quality_stats['queries_with_5_results']}")
    print(f"   Запросов с 3+ качественными результатами: {quality_stats['queries_with_3_plus_results']}")
    print(f"   Запросов с 0 качественных результатов: {quality_stats['queries_with_0_results']}")
    print(f"   Всего качественных результатов: {quality_stats['total_quality_results']}")
    print(f"   Среднее качественных результатов на запрос: {quality_stats['total_quality_results']/quality_stats['total_queries']:.2f}")
    
    return df_csv, df_excel, results, quality_stats

def main():
    print("=" * 70)
    print("УЛУЧШЕННЫЙ ПОИСК - СОРТИРОВКА ПО ID")
    print("=" * 70)
    
    # Загружаем данные
    try:
        df = pd.read_csv('questions_clean.csv')
        print(f"📁 Загружен CSV: {len(df)} запросов")
        print(f"🔢 Диапазон ID в исходном файле: {df['q_id'].min()} - {df['q_id'].max()}")
    except Exception as e:
        print(f"❌ Ошибка загрузки CSV: {e}")
        return
    
    # Обрабатываем с улучшенным поиском
    model_path = "local_sbert_with_triplets"
    df_csv, df_excel, results, stats = process_dataframe_improved(df, model_path, sample_size=10_000)
    
    # Показываем примеры улучшенных результатов
    if df_csv is not None and df_excel is not None:
        print(f"\n🎯 ПРИМЕРЫ УЛУЧШЕННЫХ РЕЗУЛЬТАТОВ (первые 5 по порядку):")
        
        # Показываем первые 5 запросов по порядку ID
        for i, item in enumerate(results[:5]):
            print(f"\n--- Запрос {item['q_id']} (порядковый №{i+1}) ---")
            print(f"📋 Запрос: '{item['query']}'")
            print(f"✅ Найдено качественных результатов: {item['results_count']}")
    
            for j, result in enumerate(item['results']):
                print(f"  {j+1}. [{result['score']:.4f}] {result['title']}")
                print(f"     🔗 {result['url']}")


if __name__ == "__main__":
    main()

УЛУЧШЕННЫЙ ПОИСК - СОРТИРОВКА ПО ID
📁 Загружен CSV: 6977 запросов
🔢 Диапазон ID в исходном файле: 1 - 6977
Загружаем локальную модель из: local_sbert_with_triplets
⏳ Загрузка модели...
✅ Модель загружена!
📊 Обрабатываем все 6977 запросов
🔢 Исходные данные отсортированы по q_id: 1 - 6977


Улучшенный поиск: 100%|██████████| 6977/6977 [1:17:00<00:00,  1.51it/s]


🔢 Результаты отсортированы по q_id: 1 - 6977

💾 Сохраняем улучшенные результаты...
✅ Улучшенный CSV со ссылками сохранен как: search_results_improved_links.csv
📊 Размер: 6977 строк × 17 колонок
🔢 Отсортировано по query_id: 1 - 6977
✅ Улучшенный Excel с текстами сохранен как: search_results_improved_texts.xlsx
📊 Размер: 6977 строк × 28 колонок
🔢 Отсортировано по query_id: 1 - 6977

📊 ДЕТАЛЬНАЯ СТАТИСТИКА КАЧЕСТВА:
   Обработано запросов: 6977
   Запросов с 5 качественными результатами: 6973
   Запросов с 3+ качественными результатами: 3
   Запросов с 0 качественных результатов: 0
   Всего качественных результатов: 34879
   Среднее качественных результатов на запрос: 5.00

🎯 ПРИМЕРЫ УЛУЧШЕННЫХ РЕЗУЛЬТАТОВ (первые 5 по порядку):

--- Запрос 1 (порядковый №1) ---
📋 Запрос: 'Номер счета'
✅ Найдено качественных результатов: 5
  1. [0.5950] Платежи в рублях
     🔗 https://alfabank.ru/sme/cashmanagment/payments/
  2. [0.5904] Где посмотреть все мои счета?
     🔗 https://alfabank.ru/help/t/corp